In [1]:
import numpy as np
import pandas as pd
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, SubsetRandomSampler
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch .cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
df = pd.read_excel("C:/Users/manis/Downloads/data.xlsx")
df

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90
...,...,...,...,...,...
47835,15.12,48.92,1011.80,72.93,462.59
47836,33.41,77.95,1010.30,59.72,432.90
47837,15.99,43.34,1014.20,78.66,465.96
47838,17.65,59.87,1018.58,94.65,450.93


In [4]:
class NeuralNetwork (nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(4,32),
            nn.ReLU(),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Linear(16,8),
            nn.ReLU(),
            nn.Linear(8,4),
            nn.ReLU(),
            nn.Linear(4,1),
        

        )

    def forward (self ,x_tensor):
        logits=self.linear_relu_stack(x_tensor)
        return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=4, bias=True)
    (7): ReLU()
    (8): Linear(in_features=4, out_features=1, bias=True)
  )
)


In [5]:
x=df.drop(columns=['PE'])
#y =df.iloc[:,-1]
y=df.drop(columns=['AT','V','AP','RH'])
#y=reshape(47840,1, keepdims=True)
print(x.shape)
print(y.shape)


(47840, 4)
(47840, 1)


In [6]:
x_tensor=torch.tensor(x.values,dtype=torch.float32)
y_tensor=torch.tensor(y.values,dtype=torch.float32)
#y_tensor.unsqueeze(1)
#y_tensor.reshape(47840,1,keepdims)
print(y_tensor.shape)
dataset=TensorDataset(x_tensor,y_tensor)


num_samples = len(dataset)
indices = list(range(num_samples))
split = int(num_samples * 0.8) 
train_indices, val_indices = indices[:split], indices[split:]
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)

torch.Size([47840, 1])


In [7]:
loss_fn=nn.MSELoss()
optimizer=optim.Adam(model.parameters(), lr=1e-3)

In [8]:
def train (dataloader , model  ,loss_fn , optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch ,(x,y)in enumerate(dataloader):
        x,y=x.to(device), y.to(device)

        pred=model(x)
        loss=loss_fn(pred,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch %100==0:
            loss,current=loss.item(),(batch+1)*len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



In [9]:
def test (dataloader , model , loss_fn):
    size = len(dataloader.dataset)
    num_batches=len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x_tensor, y_tensor in dataloader:
            x_tensor, y_tensor = x_tensor.to(device), y_tensor.to(device)
            pred = model(x_tensor)
            test_loss += loss_fn(pred, y_tensor).item()
            #correct += (pred.argmax(1) == y_tensor).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n  Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(val_loader, model, loss_fn)
print("Done ig")

Epoch 1
-------------------------------
loss: 202680.468750  [   32/47840]
loss: 2067.227539  [ 3232/47840]
loss: 206.368988  [ 6432/47840]
loss: 266.881012  [ 9632/47840]
loss: 282.799133  [12832/47840]
loss: 174.371918  [16032/47840]
loss: 284.479614  [19232/47840]
loss: 269.796936  [22432/47840]
loss: 175.035355  [25632/47840]
loss: 213.281494  [28832/47840]
loss: 156.343689  [32032/47840]
loss: 148.635422  [35232/47840]
Test Error: 
  Avg loss: 179.640121 

Epoch 2
-------------------------------
loss: 191.883240  [   32/47840]
loss: 178.507767  [ 3232/47840]
loss: 129.523758  [ 6432/47840]
loss: 144.981567  [ 9632/47840]
loss: 131.701126  [12832/47840]
loss: 92.085739  [16032/47840]
loss: 123.488457  [19232/47840]
loss: 78.671829  [22432/47840]
loss: 94.856674  [25632/47840]
loss: 68.101608  [28832/47840]
loss: 100.797363  [32032/47840]
loss: 57.236885  [35232/47840]
Test Error: 
  Avg loss: 51.768526 

Epoch 3
-------------------------------
loss: 58.073734  [   32/47840]
loss: 5